# Lab. 3-1 Schema Preparation-2

이 노트북에서는 아래 그림의 `4. Table Summarizer` 과정을 수행합니다. 

일반적인 Schema Linking 과정이 테이블 선택 -> 컬럼 선택으로 나눠 진행되는데, 테이블 선택이 잘못되면 후속 과정은 무의미하기 때문에 각 테이블에 대한 충분한 설명을 갖추는 것이 중요합니다.

이 노트북에서는 LLM을 활용해 테이블에 대한 자세한 설명 문서를 갖추는 과정을 시뮬레이션 합니다.

![Intro](../images/text2sql/schema-prep-1.png)


## Step 0: OpenSearch 환경 설정

In [ ]:
import sys
from libs.ssm import parameter_store

pm = parameter_store('us-west-2')
domain_endpoint = pm.get_params(key="chatbot-opensearch_domain_endpoint", enc=False)
opensearch_domain_endpoint = f"https://{domain_endpoint}"
opensearch_user_id = pm.get_params(key="chatbot-opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="chatbot-opensearch_user_password", enc=True)
print(opensearch_domain_endpoint)

## Step 1: Schema Description 및 Example Queries 로드

In [ ]:
import json 
SCHEMA_FILE_PATH = "./chinook_schema.json"
SAMPLE_QUERY_FILE_PATH = "./example_queries_temp.jsonl"

def load_schema(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        schema = json.load(file)
    return schema

def load_queries(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        queries = file.readlines()
    return queries

schema = load_schema(SCHEMA_FILE_PATH)
queries = load_queries(SAMPLE_QUERY_FILE_PATH)

## Step 2: 테이블 요약 문서 생성

다양한 정보들을 테이블 요약 문서 생성에 활용합니다. 

기본 Schema Description 문서와 Sample Query 등을 모두 활용해서 테이블 요약을 생성합니다. 

아래는 이 정보를 반영하기 위한 LLM 프롬프트 템플릿입니다.

In [ ]:
from langchain_aws import ChatBedrock

SYS_PROMPT = """
You are a data analyst that can help summarize SQL tables.
Summarize the provided table by the given context.

<instruction>
- You shall write the summary based only on the provided information, and make it as detailed as possible.
- Note that above sampled queries are only small sample of queries and thus not all possible use of tables are represented, and only some columns in the table are used.
- Do not use any adjective to describe the table. For example, the importance of the table, its comprehensiveness or if it is crucial, or who may be using it. For example, you can say that a table contains certain types of data, but you cannot say that the table contains a 'wealth' of data, or that it is 'comprehensive'.
- Do not mention about the sampled query. Only talk objectively about the type of data the table contains and its possible utilities.
- Please also include some potential usecases of the table, e.g. what kind of questions can be answered by the table, what kind of anlaysis can be done by the table, etc.
- Please provide the output in Korean.
</instruction>
"""

PROMPT_TEMPLATE = """
<table schema>
{table_schema}
</table schema>

<sample queries>
{sample_queries}
</sample queries>
"""

model_kwargs =  { 
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "system": SYS_PROMPT
}

chat_model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name='us-west-2',
    model_kwargs=model_kwargs
)

In [ ]:
def search_table_queries(queries, table_name):  # 테이블이 어떤 쿼리에 사용되었는지 검색하여 추출하는 함수입니다.
    table_name_lower = table_name.lower()
    matched_queries = []

    for line in queries:
        try:
            query_data = json.loads(line)
            if table_name_lower in query_data['query'].lower():
                matched_queries.append(query_data)
        except json.JSONDecodeError:
            print(f"Invalid JSON line: {line}")
    
    return matched_queries

#### 주어진 정보를 바탕으로 `Customer`라는 테이블에 대한 요약 문서를 추출해보겠습니다.

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

table_name = 'Customer'

matched_queries = search_table_queries(queries, table_name)
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
chain = prompt | chat_model | StrOutputParser()

table_summary = chain.invoke({"table_schema": schema[0]['Customer'], "sample_queries": matched_queries})
print(table_summary)

#### 아래는 이 작업을 Schema Description 내 모든 테이블을 대상으로 수행하는 코드입니다 (약 2-3분 소요됩니다)

In [ ]:
import os

OUTPUT_FILE_PATH1 = "./chinook_detailed_schema_temp.json"

with open(OUTPUT_FILE_PATH1, 'w', encoding='utf-8') as output_file:
    output_file.write('[\n')

def summarize_table(table_name, table_data, queries, chain):
    table_summary = chain.invoke({"table_schema": table_data, "sample_queries": queries})
    table_data['table_summary'] = table_summary 
    summary_output = {table_name: table_data}
    return summary_output
    
for table_info in schema:
    for table_name, table_data in table_info.items():
        globals()[table_name] = table_data
        matched_queries = search_table_queries(queries, table_name)
        prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
        chain = prompt | chat_model | StrOutputParser()

        table_summary = summarize_table(table_name, table_data, matched_queries, chain)
        
        with open(OUTPUT_FILE_PATH1, 'a', encoding='utf-8') as output_file:
            output_file.write(json.dumps(table_summary, ensure_ascii=False, indent=4) + ',\n')

with open(OUTPUT_FILE_PATH1, 'rb+') as output_file:
    output_file.seek(-2, os.SEEK_END) 
    output_file.truncate() 
    output_file.write(b'\n]')

이제 `text-to-sql-bedrock/lab3_text2sql_schema_preparation/chinook_detailed_schema_temp.json` 파일을 열어보면, table_summary가 스키마 문서에 추가되어 있습니다.

위와 같이, 1) 테이블에 어떤 컬럼들이 있는지, 2) 어떤 용도로 활용되는지에 대한 자세한 정보를 LLM에 전달하는 것은 올바른 테이블 선택에 도움이 됩니다.

하지만, 테이블 요약이 너무 길어졌을 때 모든 테이블의 요약 정보를 LLM에 전달할 수 없으므로, 테이블 요약 정보 역시 벡터 유사도 검색으로 탐색하는 것이 좋습니다.

## Step 3: 테이블 요약 문서를 벡터 임베딩으로 변환하여 OpenSearch에 저장

이 Step은 `1.sample_queries.ipynb`에서 수행한 샘플 쿼리 저장과정과 유사하게 진행됩니다.

In [ ]:
import yaml
from opensearchpy import OpenSearch, RequestsHttpConnection
INDEX_NAME = "schema_description"

def load_opensearch_config():
    with open("../libs/opensearch.yml", 'r', encoding='utf-8') as file:
        return yaml.safe_load(file)

def init_opensearch(config):
    mapping = {"settings": config['settings'], "mappings": config['mappings-schema']}
    endpoint = opensearch_domain_endpoint
    http_auth = (opensearch_user_id, opensearch_user_password)

    os_client = OpenSearch(
            hosts=[{'host': endpoint.replace("https://", ""),'port': 443}],
            http_auth=http_auth, 
            use_ssl=True,
            verify_certs=True,
            timeout=300,
            connection_class=RequestsHttpConnection
    )

    create_os_index(os_client, mapping)
    return os_client

def create_os_index(os_client, mapping):
    exists = os_client.indices.exists(INDEX_NAME)

    if exists:
        os_client.indices.delete(index=INDEX_NAME)
        print("Existing index has been deleted. Create new one.")
    else:
        print("Index does not exist, Create one.")

    os_client.indices.create(INDEX_NAME, body=mapping)

config = load_opensearch_config()
os_client = init_opensearch(config)

In [ ]:
from langchain_community.embeddings import BedrockEmbeddings

emb_model = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", region_name='us-west-2', model_kwargs={"dimensions":1024}) 
OUTPUT_FILE_PATH2 = "./chinook_detailed_schema.json"

def embedding_summary(emb_model):
    with open(OUTPUT_FILE_PATH1, 'r', encoding='utf-8') as input_file:
        data_list = json.load(input_file)

    for data in data_list:
        table_name = list(data.keys())[0]
        table_summary = data[table_name]["table_summary"]
        data[table_name]["table_summary_v"] = emb_model.embed_query(table_summary)
    
    with open(OUTPUT_FILE_PATH2, 'w', encoding='utf-8') as output_file:
        json.dump(data_list, output_file, ensure_ascii=False, indent=4)

embedding_summary(emb_model)

#### 이제 `text-to-sql-bedrock/lab3_text2sql_schema_preparation/chinook_detailed_schema_temp.json` 파일을 열어보면, 
#### table_summary 및 이에 대한 임베딩이 스키마 문서에 추가되어 있습니다.

In [ ]:
def load_detailed_schema_descriptions(os_client):

    with open(OUTPUT_FILE_PATH2, 'r') as file:
        schema_data = json.load(file)

    bulk_data = []
    for table in schema_data:
        for table_name, table_info in table.items():
            table_doc = {
                "table_name": table_name,
                "table_desc": table_info["table_desc"],
                "columns": [{"col_name": col["col"], "col_desc": col["col_desc"]} for col in table_info["cols"]],
                "table_summary": table_info["table_summary"],
                "table_summary_v": table_info["table_summary_v"]
            }
            bulk_data.append({"index": {"_index": INDEX_NAME, "_id": table_name}})
            bulk_data.append(table_doc)
    
    bulk_data_str = '\n'.join(json.dumps(item) for item in bulk_data) + '\n'

    response = os_client.bulk(body=bulk_data_str)
    if response["errors"]:
        print("There were errors during bulk indexing:")
        for item in response["items"]:
            if 'index' in item and item['index']['status'] >= 400:
                print(f"Error: {item['index']['error']['reason']}")
    else:
        print("Bulk-inserted all items successfully.")

load_detailed_schema_descriptions(os_client)

#### 이제 OpenSearch에 스키마 정보의 저장을 완료했습니다.